In [2]:
from keras.models import Sequential, load_model
from keras.layers import Convolution2D, Dense, Flatten, Dropout, MaxPooling2D, BatchNormalization
from keras.optimizers import Adam, SGD
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import CSVLogger, ModelCheckpoint, ReduceLROnPlateau
from keras.utils import np_utils

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.metrics import fbeta_score
from sklearn.externals import joblib

import numpy as np
import cPickle as pickle
import pandas as pd
import cv2
from PIL import Image
import matplotlib.pyplot as plt
%matplotlib inline
import glob
import os
import sys

Using Theano backend.
WARNING (theano.sandbox.cuda): The cuda backend is deprecated and will be removed in the next release (v0.10).  Please switch to the gpuarray backend. You can get more information about how to switch at this URL:
 https://github.com/Theano/Theano/wiki/Converting-to-the-new-gpu-back-end%28gpuarray%29

Using gpu device 0: GeForce GTX 950M (CNMeM is enabled with initial size: 70.0% of memory, cuDNN 5005)


In [3]:
cur = os.getcwd()
data_path = cur + '\\..\\data\\'

all_labels = ['agriculture', 'artisinal_mine', 'bare_ground', 'blooming', 'blow_down', 'clear', 'cloudy', 'conventional_mine', 'cultivation', 'habitation', 'haze', 'partly_cloudy', 'primary', 'road', 'selective_logging', 'slash_burn', 'water']
labels = ['clear', 'cloudy', 'haze', 'partly_cloudy']
train_csv = pd.read_csv(os.path.join(data_path, 'train.csv'))
train_labels = train_csv.tags.str.split(' ')
train_labels = [[lab for lab in labelset if lab in labels] for labelset in train_labels]
train_labels = MultiLabelBinarizer().fit_transform(train_labels)
print(train_labels.shape)

(40479L, 4L)


In [49]:
jpg_path = data_path + 'train-jpg\\*.jpg'
filelist = glob.glob(jpg_path)
train_data = np.array([np.array(Image.open(fname)) for fname in filelist])
print(train_data.shape)

(40479L, 256L, 256L, 4L)


In [51]:
train_data = np.transpose(train_data, (0, 3, 1, 2))

#train_data = train_data.astype('float32')
#train_data = train_data / 255.0
#with open('train_jpg.pkl', 'wb') as outfile:
    #pickle.dump(train_data, outfile, pickle.HIGHEST_PROTOCOL)


In [67]:
X_train = train_data[1:30479, :, :, :]
y_train= train_labels[1:30479, :]
X_val = train_data[30479::, :, :, :]
y_val = train_labels[30479::, :]

In [68]:
model = Sequential()

model.add(BatchNormalization(input_shape=[4, 256, 256], axis=1))

model.add(Convolution2D(32, 3, 3, input_shape=[4, 256, 256], activation='relu'))
model.add(BatchNormalization(axis=1))
model.add(Convolution2D(32, 3, 3, activation='relu'))
model.add(BatchNormalization(axis=1))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Convolution2D(64, 3, 3, activation='relu'))
model.add(BatchNormalization(axis=1))
model.add(Convolution2D(64, 3, 3, activation='relu'))
model.add(BatchNormalization(axis=1))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Convolution2D(128, 3, 3, activation='relu'))
model.add(BatchNormalization(axis=1))
#model.add(Convolution2D(128, 3, 3, activation='relu'))
#model.add(BatchNormalization(axis=1))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Flatten())
model.add(Dense(512, activation='relu'))
model.add(Dense(len(labels), activation='softmax'))

model.compile(loss='categorical_crossentropy',
             optimizer=Adam(),
             metrics=['accuracy', 'categorical_crossentropy'])

model.summary()

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
batchnormalization_7 (BatchNorma (None, 4, 256, 256)   8           batchnormalization_input_2[0][0] 
____________________________________________________________________________________________________
convolution2d_6 (Convolution2D)  (None, 32, 254, 254)  1184        batchnormalization_7[0][0]       
____________________________________________________________________________________________________
batchnormalization_8 (BatchNorma (None, 32, 254, 254)  64          convolution2d_6[0][0]            
____________________________________________________________________________________________________
convolution2d_7 (Convolution2D)  (None, 32, 252, 252)  9248        batchnormalization_8[0][0]       
___________________________________________________________________________________________

In [69]:
csv_logger = CSVLogger('run4_adam.csv')
lr_plateau = ReduceLROnPlateau(monitor='val_loss', patience=1, verbose=1, factor=0.5)
checkpoint = ModelCheckpoint(filepath='E:/KI/Master/MLIP/Brainforest/WeatherTest/model.{epoch:02d}-{val_loss:.2f}.hdf5',
                             verbose=1, save_best_only=True)

model.fit(X_train, y_train, validation_data=(X_val, y_val), nb_epoch=25, batch_size=8, verbose=1, callbacks=[csv_logger, lr_plateau, checkpoint])

Train on 30478 samples, validate on 10000 samples
Epoch 1/25
30478/30478 [==============================] - 2594s - loss: 5.3876 - acc: 0.6654 - categorical_crossentropy: 5.3876 - val_loss: 4.9106 - val_acc: 0.6952 - val_categorical_crossentropy: 4.9106
Epoch 2/25
30478/30478 [==============================] - 2607s - loss: 4.9214 - acc: 0.6946 - categorical_crossentropy: 4.9214 - val_loss: 4.8725 - val_acc: 0.6977 - val_categorical_crossentropy: 4.8725
Epoch 3/25
30478/30478 [==============================] - 2601s - loss: 4.8992 - acc: 0.6960 - categorical_crossentropy: 4.8992 - val_loss: 4.8725 - val_acc: 0.6977 - val_categorical_crossentropy: 4.8725
Epoch 4/25
30472/30478 [============================>.] - ETA: 0s - loss: 4.9883 - acc: 0.6905 - categorical_crossentropy: 4.9883
Epoch 00003: reducing learning rate to 0.000500000023749.
Epoch 00003: val_loss did not improve
30478/30478 [==============================] - 2589s - loss: 4.9873 - acc: 0.6905 - categorical_crossentropy: 4.

KeyboardInterrupt: 